In [3]:
!pip install tensorflow==2.0
!pip install transformers==2.11.0
!pip list | grep -E 'transformers|tokenizers'

     |████████████████████████████████| 86.3MB 70kB/s 
     |████████████████████████████████| 450kB 33.1MB/s 
     |████████████████████████████████| 3.8MB 42.0MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=63b4be90cd8c2a111520a75a7d9d971f2f9f6f72b454c2bc24a2a94812876934
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstal

In [2]:
!nvidia-smi

Thu Nov 19 09:55:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [4]:
!mkdir Wordpiece_distilBert_Tokenizer

mkdir: cannot create directory ‘Wordpiece_distilBert_Tokenizer’: File exists


In [5]:
from tokenizers import BertWordPieceTokenizer

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer()

# Then train it!
tokenizer.train(files = ["drive/MyDrive/tech_corpus_wikiAndTextBook.txt"],min_frequency=2)



In [11]:
# And finally save it somewhere 
tokenizer.save("/content/Wordpiece_distilBert_Tokenizer")
#tokenizer.save_model('./Wordpiece_distilBert_Tokenizer')

['/content/Wordpiece_distilBert_Tokenizer/vocab.txt']

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
tokenizer.encode('java is a programming language').tokens

['java', 'is', 'a', 'programming', 'language']

In [13]:
from transformers import DistilBertTokenizerFast

trained_tokenizer = DistilBertTokenizerFast.from_pretrained("./Wordpiece_distilBert_Tokenizer", do_lower_case=True, add_special_tokens=True,
                                                       pad_to_max_length=True)

In [14]:
from transformers import DistilBertConfig
config = DistilBertConfig()

In [15]:
from transformers import DistilBertModel

model = DistilBertModel(config=config)

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=trained_tokenizer,
    file_path="drive/MyDrive/tech_corpus_wikiAndTextBook.txt",
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=trained_tokenizer)

In [ ]:
!mkdir Wordpiece_distilBert

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./Wordpiece_distilBert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# New Section

In [ ]:
model.num_parameters()

/Users/meherabhishek/PycharmProjects/AutoEvaluation/bertenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


66362880

In [ ]:
%%time
trainer.train()

TypeError: forward() got an unexpected keyword argument 'labels'

### Sanjeev changes

In [6]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv("sample_data_test.csv")
df.head()

,Unnamed: 0,correct_ans,candidate_ans,pretrained_glove_score_0.1.0,tl_glove_score_1.0.0,tl_glove_score_1.1.0,correct_ans_vec,candidate_ans_vec
0,0,Machine learning is an application of artifici...,Machine Learning is the science of getting com...,98.242176,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['machine', 'learning', 'is', 'the', 'science'..."
1,1,Machine learning is an application of artifici...,Machine Learning (ML) can be explained as auto...,98.644829,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['machine', 'learning', '(ml)', 'can', 'be', '..."
2,2,Machine learning is an application of artifici...,Data science is an inter-disciplinary field th...,97.109884,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['data', 'science', 'is', 'an', 'inter-discipl..."
3,3,Machine learning is an application of artifici...,software engineers are really good in coding t...,94.816715,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['software', 'engineers', 'are', 'really', 'go..."
4,4,Machine learning is an application of artifici...,Joint Entrance Examination – Advanced (JEE-Adv...,92.076212,-0.0,-0.0,"['machine', 'learning', 'is', 'an', 'applicati...","['joint', 'entrance', 'examination', '–', 'adv..."


In [9]:
df = df.iloc[:,1:3]
df

,correct_ans,candidate_ans
0,Machine learning is an application of artifici...,Machine Learning is the science of getting com...
1,Machine learning is an application of artifici...,Machine Learning (ML) can be explained as auto...
2,Machine learning is an application of artifici...,Data science is an inter-disciplinary field th...
3,Machine learning is an application of artifici...,software engineers are really good in coding t...
4,Machine learning is an application of artifici...,Joint Entrance Examination – Advanced (JEE-Adv...


In [10]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [11]:
tokenized = df["candidate_ans"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
tokenized.shape

(6,)

In [28]:
df["candidate_ans"][3]

'software engineers are really good in coding they are a good team player and good in mathematics'

In [16]:
tokenized[3]

[101,
 4007,
 6145,
 2024,
 2428,
 2204,
 1999,
 16861,
 2027,
 2024,
 1037,
 2204,
 2136,
 2447,
 1998,
 2204,
 1999,
 5597,
 102]

In [17]:
tokenized[5]= tokenizer.encode(text, add_special_tokens=True)

In [19]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [20]:
np.array(padded).shape

(6, 183)

In [21]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(6, 183)

In [22]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [23]:
features = last_hidden_states[0][:,0,:].numpy() 

In [24]:
features.shape

(6, 768)

In [12]:
text ="The president of India, officially the President of the Republic of India (IAST: Bhārat kē Rāṣhṭrapati), is the ceremonial head of state of India and the Commander-in-chief of the Indian Armed Forces.The president is indirectly elected by an electoral college comprising the Parliament of India (both houses) and the legislative assemblies of each of India's states and territories, who themselves are all directly elected."

In [25]:
print(df["candidate_ans"].values[0])

Machine Learning is the science of getting computers to learn and act like humans do, and improve their learning over time in autonomous fashion, by feeding them data and information in the form of observations and real-world interactions.


In [29]:
features[0]

array([-3.08142602e-01, -1.49790034e-01, -5.20846784e-01,  2.42292330e-01,
        6.07292391e-02, -3.06847338e-02,  3.01691920e-01,  3.48404735e-01,
       -6.09287713e-03, -6.42100513e-01,  6.14608116e-02, -1.92244351e-01,
       -5.73620915e-01,  3.59506607e-01, -5.03467321e-01, -5.73552959e-02,
        9.32614952e-02,  8.70709121e-02, -2.76093632e-02,  4.06692296e-01,
       -4.62785631e-01, -1.39107943e-01, -9.08476114e-02,  5.42547345e-01,
        2.11270839e-01, -5.75832427e-01,  9.73371193e-02,  1.46942824e-01,
        4.09967333e-01, -1.04062557e-01, -2.68188179e-01,  1.85617760e-01,
       -2.71366656e-01, -4.76074189e-01,  2.66784221e-01,  9.05128382e-03,
       -1.99091449e-01, -4.56973404e-01, -1.47691473e-01,  2.84025699e-01,
       -1.69437349e-01,  1.34420440e-01, -1.92250669e-01,  7.42065609e-02,
       -3.53516132e-01, -6.43529892e-02, -3.29623890e+00,  1.65182829e-01,
       -1.93200573e-01,  3.15806246e-03, -3.01448345e-01,  2.60472819e-02,
       -1.17619514e-01,  

In [27]:
#cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(features, features))

[[1.         0.942971   0.96047986 0.81406295 0.8187834  0.7977904 ]
 [0.942971   1.0000001  0.91257393 0.8497659  0.80976015 0.75950277]
 [0.96047986 0.91257393 1.0000002  0.7982632  0.827279   0.7801221 ]
 [0.81406295 0.8497659  0.7982632  0.9999995  0.7638572  0.7189686 ]
 [0.8187834  0.80976015 0.827279   0.7638572  0.9999999  0.8490895 ]
 [0.7977904  0.75950277 0.7801221  0.7189686  0.8490895  0.99999994]]
